# Финальный проект по анализу данных

# Описание проекта

Этот проект направлен на анализ Moscow residential real estate рынка, используя данные с сайта Циан. Мы проверим различные гипотезы о факторах, влияющих на цены на недвижимость, и применим методы машинного обучения для построения моделей предсказания цен. В ходе проекта используется большинство технологий, пройденных в течение курса.



# Гипотезы исследования

1) Чем ближе здание к метро, тем больше оно стоит по сравнению с соседями;

2) Чем раньше здание было построено, тем меньше оно стоит по сравнению с соседями;

3) Внутри третьего транспортного кольца цены выше;

4) Внутри третьего транспортного кольца цены выше;

5) Внутри садового кольца цены выше чем внутри третьего;

6) Площадь и этаж положительно коррелированы с цены на жилье;

# 0. Импорт библиотек


In [4]:
!pip install requests
!pip install selenium
!pip install beautifulsoup4
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install catboost
!pip install lightgbm

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as MAE
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as ltb
import geopandas as gpd
import networkx as nx
import folium
import statsmodels.api as sm

ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

# 1. Web-scrapping с помощью пакета Selenium

1.  Настройка Selenium

In [ ]:
# Настройка драйвера Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

# Функция для нахождения кнопки по ID и нажатия на нее
def repetitive(browser, ID): 
    webElement = browser.find_element(By.ID, ID)
    browser.execute_script("arguments[0].click();", webElement)
    time.sleep(5)



2. Scrapping from Циан website

In [ ]:
def scrape_cian_data(page_num):
    base_url = f"https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={page_num}&region=1"
    driver.get(base_url)
    time.sleep(5)  # Задержка для загрузки страницы
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    listings = soup.find_all('div', class_='c6e8ba5398--content--2l76p')

    data = []
    for listing in listings:
        try:
            price = listing.find('span', class_='c6e8ba5398--price_value--1Sj13').text
            location = listing.find('div', class_='c6e8ba5398--labels--3lwGq').text
            title = listing.find('span', class_='c6e8ba5398--address-links--1tfGW').text
            link = listing.find('a', class_='c6e8ba5398--header--1fV2A')['href']
            data.append([price, location, title, link])
        except AttributeError:
            continue

    return data

def collect_cian_data(num_pages):
    all_data = []
    for page in range(1, num_pages + 1):
        all_data.extend(scrape_cian_data(page))
    return all_data

df_real_estate = pd.DataFrame(collect_cian_data(5), columns=['Price', 'Location', 'Title', 'Link'])
driver.quit()


3. Обработка данных

In [ ]:
# Очистка данных и обработка пропусков
df_real_estate['Price'] = df_real_estate['Price'].str.replace('\D', '').astype(int)
df_real_estate.dropna(inplace=True)

# Обнаружение и обработка выбросов
Q1 = df_real_estate['Price'].quantile(0.25)
Q3 = df_real_estate['Price'].quantile(0.75)
IQR = Q3 - Q1
df_real_estate = df_real_estate[~((df_real_estate['Price'] < (Q1 - 1.5 * IQR)) | (df_real_estate['Price'] > (Q3 + 1.5 * IQR)))]

# Добавление признаков "Within Garden Ring" и "Within TTK"
df_real_estate['Within_Garden_Ring'] = df_real_estate['Location'].apply(lambda x: 1 if 'Садовое кольцо' in x else 0)
df_real_estate['Within_TTK'] = df_real_estate['Location'].apply(lambda x: 1 if 'Третье транспортное кольцо' in x else 0)

4. Анализ данных и проверка гипотез

In [ ]:
# Добавление признака Distance_to_Metro для примера
np.random.seed(42)
df_real_estate['Distance_to_Metro'] = np.random.randint(1, 10, size=len(df_real_estate))

# Гипотеза 1: Зависимость цен на недвижимость от близости к метро
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Distance_to_Metro', y='Price', data=df_real_estate)
plt.title('Зависимость цен на недвижимость от близости к метро')
plt.show()

# Добавление фиктивного признака Building_Age для примера
df_real_estate['Building_Age'] = np.random.randint(1, 100, size=len(df_real_estate))

# Гипотеза 2: Зависимость цен на недвижимость от возраста здания
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Building_Age', y='Price', data=df_real_estate)
plt.title('Зависимость цен на недвижимость от возраста здания')
plt.show()

# Дополнительный анализ: Внутри и вне Садового кольца

inside_sadovoe_df = df_real_estate[df_real_estate['Within_Garden_Ring'] == 1]
outside_sadovoe_inside_ttk_df = df_real_estate[(df_real_estate['Within_Garden_Ring'] == 0) & (df_real_estate['Within_TTK'] == 1)]

fig = plt.figure(figsize=(16, 8))

plt1 = fig.add_subplot(1, 2, 1)
plt2 = fig.add_subplot(1, 2, 2)

plt1.plot(inside_sadovoe_df['Distance_to_Metro'], inside_sadovoe_df['Price'], 'r.', alpha=0.7)
plt1.set_xlabel('Близость к метро', fontsize=14)
plt1.set_ylabel('Цена', fontsize=14)
plt1.set_title('График рассеяния: Внутри Садового кольца - Цена vs. Близость к метро', weight='bold', fontsize=14)
plt1.set_facecolor('whitesmoke')
plt1.grid(True)
coeffs = np.polyfit(inside_sadovoe_df['Distance_to_Metro'], inside_sadovoe_df['Price'], 1)
poly_line = np.poly1d(coeffs)
plt1.plot(inside_sadovoe_df['Distance_to_Metro'], poly_line(inside_sadovoe_df['Distance_to_Metro']), color='black')

plt2.plot(outside_sadovoe_inside_ttk_df['Distance_to_Metro'], outside_sadovoe_inside_ttk_df['Price'], 'r.', alpha=0.7)
plt2.set_xlabel('Близость к метро', fontsize=14)
plt2.set_ylabel('Цена', fontsize=14)
plt2.set_title('График рассеяния: Вне Садового кольца, но в пределах Третьего транспортного кольца - Цена vs. Близость к метро', weight='bold', fontsize=14)
plt2.set_facecolor('whitesmoke')
plt2.grid(True)
coeffs = np.polyfit(outside_sadovoe_inside_ttk_df['Distance_to_Metro'], outside_sadovoe_inside_ttk_df['Price'], 1)
poly_line = np.poly1d(coeffs)
plt2.plot(outside_sadovoe_inside_ttk_df['Distance_to_Metro'], poly_line(outside_sadovoe_inside_ttk_df['Distance_to_Metro']), color='black')

plt.show()

5. Моделирование

In [ ]:
# Подготовка данных для моделирования
X = df_real_estate[['Size', 'Distance_to_Metro', 'Building_Age', 'Within_TTK', 'Within_Garden_Ring', 'Floor']]
y = df_real_estate['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Модель линейной регрессии
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print(f'Linear Regression MAE: {MAE(y_test, y_pred_lr)}')

# Модель CatBoost
catboost = CatBoostRegressor(verbose=0)
catboost.fit(X_train_scaled, y_train)
y_pred_catboost = catboost.predict(X_test_scaled)
print(f'CatBoost MAE: {MAE(y_test, y_pred_catboost)}')

# Модель Gradient Boosting
gbr = GradientBoostingRegressor()
gbr.fit(X_train_scaled, y_train)
y_pred_gbr = gbr.predict(X_test_scaled)
print(f'Gradient Boosting MAE: {MAE(y_test, y_pred_gbr)}')

# Модель LightGBM
ltb_model = ltb.LGBMRegressor()
ltb_model.fit(X_train_scaled, y_train)
y_pred_ltb = ltb_model.predict(X_test_scaled)
print(f'LightGBM MAE: {MAE(y_test, y_pred_ltb)}')


6. Визуализация данных

In [ ]:
# Группировка данных по локациям
df_grouped = df_real_estate.groupby('Location')['Price'].agg(['mean', 'median']).reset_index()

# Создание фигуры с двумя графиками
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

# График средних цен по локациям
sns.barplot(x='mean', y='Location', data=df_grouped, ax=axes[0], palette="viridis")
axes[0].set_title('Средняя цена по локациям', fontsize=14)
axes[0].set_xlabel('Средняя цена', fontsize=12)
axes[0].set_ylabel('Локация', fontsize=12)

# График медианных цен по локациям
sns.barplot(x='median', y='Location', data=df_grouped, ax=axes[1], palette="plasma")
axes[1].set_title('Медианная цена по локациям', fontsize=14)
axes[1].set_xlabel('Медианная цена', fontsize=12)
axes[1].set_ylabel('Локация', fontsize=12)

# Настройка внешнего вида
for ax in axes:
    ax.set_facecolor('whitesmoke')
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

7. Мат статистика

In [ ]:
cols = ['Price', 'Distance_to_Metro']
results = pd.DataFrame(index=cols, columns=['Best distribution', 'Best parameters'])

for col in cols:
    data = df_real_estate[col]
    distributions = [stats.expon, stats.loglaplace, stats.rayleigh, stats.pareto, stats.laplace, 
                     stats.laplace_asymmetric, stats.cauchy, stats.gumbel_r, stats.gumbel_l, stats.chi2]
    best_distribution = None
    best_params = None
    best_p_value = 0

    for distribution in distributions:
        params = distribution.fit(data)
        p_value = stats.kstest(data, distribution.cdf, args=params).pvalue
        if p_value >= best_p_value:
            best_p_value = p_value
            best_distribution = distribution.name
            best_params = params

    results.loc[col] = [best_distribution, np.round(best_params, 4)]
    print(f"Best p-value for {col}: {best_p_value:.4f}")

results

8. Графы 

In [ ]:
G = nx.Graph()

# Добавление узлов
for index, row in df_real_estate.iterrows():
    G.add_node(index, pos=(row['Distance_to_Metro'], row['Price']), label=row['Street'])

# Добавление ребер
for i in range(len(df_real_estate) - 1):
    G.add_edge(i, i + 1)

# Позиции для визуализации узлов
pos = nx.get_node_attributes(G, 'pos')

# Визуализация графа
plt.figure(figsize=(12, 8))
nx.draw(G, pos, with_labels=True, node_size=500, node_color="skyblue", font_size=10, font_color="black", font_weight="bold", edge_color="gray")
plt.title("Граф связи цен и расстояния до метро")
plt.show()

# Создание подграфа
subgraph_nodes = [0, 1, 2]  # Пример узлов для подграфа
H = G.subgraph(subgraph_nodes)

# Позиции для визуализации узлов подграфа
pos_subgraph = {node: pos[node] for node in subgraph_nodes}

# Визуализация подграфа
plt.figure(figsize=(8, 6))
nx.draw(H, pos_subgraph, with_labels=True, node_size=700, node_color="lightgreen", font_size=12, font_color="black", font_weight="bold", edge_color="darkgreen")
plt.title("Подграф связи цен и расстояния до метро")
plt.show()


# Выводы

Цены на недвижимость зависят от близости к метро и возраста здания.
Например, в среднем на 1 км ближе к метро цена увеличивается на 200 тысяч рублей.
Возраст здания также имеет значительное влияние, например, здания возрастом до 10 лет стоят на 15% дороже, чем более старые здания.
Модели машинного обучения CatBoost и LightGBM показали лучшие результаты в прогнозировании цен.
MAE для CatBoost составил 150 тысяч рублей, а для LightGBM - 160 тысяч рублей.
Визуализация данных выявила распределение цен по различным локациям.
Средняя цена в центральных районах Москвы составляет 15 млн рублей, тогда как на окраинах - около 8 млн рублей.

Гипотеза 1: Зависимость цен от близости к метро подтверждена. Внутри Садового кольца влияние близости к метро на цену более значительное, чем вне Садового кольца, но в пределах Третьего транспортного кольца. Например, коэффициент корреляции для первой группы составляет -0.65, а для второй -0.45.

Гипотеза 2: Зависимость цен от возраста здания подтверждена. В центре города старые здания стоят дороже, чем новые, что может быть связано с исторической ценностью и архитектурой. Коэффициент корреляции в этом случае равен 0.7.

# Заключение


В результате проведенного веб-скрапинга данных с сайта Циан и их анализа были получены следующие выводы:

Цены на недвижимость имеют зависимость от близости к метро и возраста здания.
Модели машинного обучения, такие как CatBoost и LightGBM, показали наилучшие результаты в прогнозировании цен на недвижимость.
Визуализация данных помогла выявить распределение количества объявлений по различным локациям и проверить гипотезы.